In [109]:
import os
import pandas as pd

folder = r"X:\SPR\Inventory Model Master Data"
all_products = set()

for file in os.listdir(folder):
    if file.endswith('.xlsx') or file.endswith('.xls'):
        path = os.path.join(folder, file)
        try:
            df = pd.read_excel(path)
            if 'Product' in df.columns:
                products = df['Product'].dropna().astype(str).unique()
                all_products.update(products)
        except Exception as e:
            print(f"Error reading {file}: {e}")

# Convert to sorted list and print or save
product_list = sorted(all_products)
print(product_list)

# Optionally, save to a new Excel file:
pd.DataFrame({'Product': product_list}).to_excel('unique_products.xlsx', index=False)

['0', '000263319', '0004092', '0004092FA', '0005480', '0005480FA', '0005480XXXC001', '000654327', '000654335', '000672055', '0026-2-00057A', '0026-2-00059A', '0026-2-00060A', '00263269', '00263277', '00263301', '0027/025112', '00281063', '00281071', '0086/046300', '0086/046316', '008915048', '008915049', '008915053', '009311045', '01596704', '0212773', '0231-M-4-789-2', '0333-CFR', '0333-CFR-S', '0333-HR-S', '04.MM.1233', '058500017', '058500018', '060-05-011MK1', '060-05-011MK2', '060-10-002MK1', '060-10-002MK2', '060-10-002MK3', '061-M-306MK1', '061-M-306MK10', '061-M-306MK2', '061-M-306MK3', '061-M-306MK4', '061-M-306MK5', '061-M-306MK6', '061-M-306MK7', '061-M-306MK8', '061-M-306MK9', '063-M-445ALH', '063-M-445ARH', '063-M-445BLH', '063-M-445BRH', '063-M-445CLH', '063-M-445CRH', '075-125', '0777-CFR', '078-M', '080500038AH', '083-M-1032MKA', '08915063', '08915063-PATTERN', '08915065', '08915065-PATTERN', '08915067', '08915067-PATTERN', '08926030', '1.504856-SC', '1.504857-SC', '1.5

 Connect to SQL Server

In [110]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

server = 'BKGCC-SQL'
database = 'Bradken_Data_Warehouse'
driver = 'ODBC Driver 17 for SQL Server'
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    "Trusted_Connection=yes;"
)
params = urllib.parse.quote_plus(conn_str)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

. Prepare Your Product List and Warehouses


In [111]:
# Assuming product_list is already defined as in your previous code
warehouse_codes = ['H53', 'I92', '12A', 'M61', '235', '261']

 Build the SQL Query

In [112]:
# Dates: last day of each month from Mar 23 to Jul 24
import pandas as pd
from datetime import datetime

date_range = pd.date_range('2023-03-31', '2024-07-31', freq='M')
date_strs = "', '".join(d.strftime('%Y-%m-%d') for d in date_range)

product_strs = "', '".join(product_list)

warehouse_strs = "', '".join(warehouse_codes)

# ...existing code...
query = f"""
SELECT 
    w.WarehouseCode,
    p.ProductKey,
    d.DateValue,
    iw.WIPQty,
    p.DressMass,
    (iw.WIPQty * p.DressMass / 1000.0) AS WIPMass
FROM [PowerBI].[Work In Progress]  iw
LEFT JOIN PowerBI.Products p ON iw.skProductId = p.skProductId
LEFT JOIN PowerBI.Warehouse w ON iw.skWarehouseId = w.skWarehouseId
LEFT JOIN PowerBI.Dates d ON iw.skReportDateId = d.skDateId
WHERE 
    w.WarehouseCode IN ('{warehouse_strs}')
    AND p.ProductKey IN ('{product_strs}')
    AND d.DateValue IN ('{date_strs}')
    AND iw.WIPValue > 0
"""
# ...existing code...

C:\Users\aali\AppData\Local\Temp\ipykernel_3220\2309725788.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range('2023-03-31', '2024-07-31', freq='M')


Fetch Data

In [113]:
df = pd.read_sql(query, engine)
print(df.head())

  WarehouseCode    ProductKey   DateValue  WIPQty  DressMass   WIPMass
0           H53   1711-05-01D  2023-03-31     1.0     133.81   0.13381
1           H53   1711-05-01D  2023-03-31    34.0     133.81   4.54954
2           H53   1711-05-01D  2023-03-31     1.0     133.81   0.13381
3           H53   1711-05-01D  2023-03-31    16.0     133.81   2.14096
4           H53  1733-204-08F  2023-03-31    28.0    2992.80  83.79840


In [114]:


# Export DataFrame to CSV
df.to_csv('wip_data_export.csv', index=False)
# ...existing code...

1. Connect to the new server

In [115]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

server = 'bknew-sql02'
database = 'Bradken_Data_Warehouse'
driver = 'ODBC Driver 17 for SQL Server'
conn_str = (
    f"DRIVER={driver};"
    f"SERVER={server};"
    f"DATABASE={database};"
    "Trusted_Connection=yes;"
)
params = urllib.parse.quote_plus(conn_str)
engine_epicor = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

2. Prepare your product list and date ranges

In [116]:
from datetime import timedelta

# Assuming product_list is already defined
product_strs = "', '".join(product_list)

site_offsets = {
    'XUZ1': 39,
    'WUN1': 39,
    'WOD1': 32,
    'MER1': 18,
    'MTJ1': 26,
    'COI2': 39
}
date_range = pd.date_range('2024-08-31', '2025-06-30', freq='M')

mapping_rows = []
for site, offset in site_offsets.items():
    for d in date_range:
        period_end = d.strftime('%Y-%m-%d')
        period_start = (d - pd.Timedelta(days=offset-1)).strftime('%Y-%m-%d')
        mapping_rows.append(f"('{site}', '{period_end}', '{period_start}')")

values_clause = ",\n    ".join(mapping_rows)
print(values_clause)

('XUZ1', '2024-08-31', '2024-07-24'),
    ('XUZ1', '2024-09-30', '2024-08-23'),
    ('XUZ1', '2024-10-31', '2024-09-23'),
    ('XUZ1', '2024-11-30', '2024-10-23'),
    ('XUZ1', '2024-12-31', '2024-11-23'),
    ('XUZ1', '2025-01-31', '2024-12-24'),
    ('XUZ1', '2025-02-28', '2025-01-21'),
    ('XUZ1', '2025-03-31', '2025-02-21'),
    ('XUZ1', '2025-04-30', '2025-03-23'),
    ('XUZ1', '2025-05-31', '2025-04-23'),
    ('XUZ1', '2025-06-30', '2025-05-23'),
    ('WUN1', '2024-08-31', '2024-07-24'),
    ('WUN1', '2024-09-30', '2024-08-23'),
    ('WUN1', '2024-10-31', '2024-09-23'),
    ('WUN1', '2024-11-30', '2024-10-23'),
    ('WUN1', '2024-12-31', '2024-11-23'),
    ('WUN1', '2025-01-31', '2024-12-24'),
    ('WUN1', '2025-02-28', '2025-01-21'),
    ('WUN1', '2025-03-31', '2025-02-21'),
    ('WUN1', '2025-04-30', '2025-03-23'),
    ('WUN1', '2025-05-31', '2025-04-23'),
    ('WUN1', '2025-06-30', '2025-05-23'),
    ('WOD1', '2024-08-31', '2024-07-31'),
    ('WOD1', '2024-09-30', '2024-08-30

C:\Users\aali\AppData\Local\Temp\ipykernel_3220\2433441125.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_range = pd.date_range('2024-08-31', '2025-06-30', freq='M')


GitHub Copilot
Here’s how you can approach this in Python and SQL, step by step:

1. Connect to the new server
2. Prepare your product list and date ranges
3. Build the SQL query

In [122]:
# Build CASE statement for date offsets using hp.TapTime instead of d.DateValue
case_stmt = "CASE s.SiteName "
for site, offset in site_offsets.items():
    case_stmt += f"WHEN '{site}' THEN DATEADD(day, -{offset}, CONVERT(date, hp.TapTime)) "
case_stmt += "END"

# ...existing code...
query_epicor = f"""
WITH PeriodMap AS (
    SELECT *
    FROM (VALUES
        ('XUZ1', '2024-08-31', '2024-07-23'),
('XUZ1', '2024-09-30', '2024-08-22'),
('XUZ1', '2024-10-31', '2024-09-22'),
('XUZ1', '2024-11-30', '2024-10-22'),
('XUZ1', '2024-12-31', '2024-11-22'),
('XUZ1', '2025-01-31', '2024-12-23'),
('XUZ1', '2025-02-28', '2025-01-20'),
('XUZ1', '2025-03-31', '2025-02-20'),
('XUZ1', '2025-04-30', '2025-03-22'),
('XUZ1', '2025-05-31', '2025-04-22'),
('XUZ1', '2025-06-30', '2025-05-22'),
('WUN1', '2024-08-31', '2024-07-23'),
('WUN1', '2024-09-30', '2024-08-22'),
('WUN1', '2024-10-31', '2024-09-22'),
('WUN1', '2024-11-30', '2024-10-22'),
('WUN1', '2024-12-31', '2024-11-22'),
('WUN1', '2025-01-31', '2024-12-23'),
('WUN1', '2025-02-28', '2025-01-20'),
('WUN1', '2025-03-31', '2025-02-20'),
('WUN1', '2025-04-30', '2025-03-22'),
('WUN1', '2025-05-31', '2025-04-22'),
('WUN1', '2025-06-30', '2025-05-22'),
('WOD1', '2024-08-31', '2024-07-30'),
('WOD1', '2024-09-30', '2024-08-29'),
('WOD1', '2024-10-31', '2024-09-29'),
('WOD1', '2024-11-30', '2024-10-29'),
('WOD1', '2024-12-31', '2024-11-29'),
('WOD1', '2025-01-31', '2024-12-30'),
('WOD1', '2025-02-28', '2025-01-27'),
('WOD1', '2025-03-31', '2025-02-27'),
('WOD1', '2025-04-30', '2025-03-29'),
('WOD1', '2025-05-31', '2025-04-29'),
('WOD1', '2025-06-30', '2025-05-29'),
('MER1', '2024-08-31', '2024-08-13'),
('MER1', '2024-09-30', '2024-09-12'),
('MER1', '2024-10-31', '2024-10-13'),
('MER1', '2024-11-30', '2024-11-12'),
('MER1', '2024-12-31', '2024-12-13'),
('MER1', '2025-01-31', '2025-01-13'),
('MER1', '2025-02-28', '2025-02-10'),
('MER1', '2025-03-31', '2025-03-13'),
('MER1', '2025-04-30', '2025-04-12'),
('MER1', '2025-05-31', '2025-05-13'),
('MER1', '2025-06-30', '2025-06-12'),
('MTJ1', '2024-08-31', '2024-08-05'),
('MTJ1', '2024-09-30', '2024-09-04'),
('MTJ1', '2024-10-31', '2024-10-05'),
('MTJ1', '2024-11-30', '2024-11-04'),
('MTJ1', '2024-12-31', '2024-12-05'),
('MTJ1', '2025-01-31', '2025-01-05'),
('MTJ1', '2025-02-28', '2025-02-02'),
('MTJ1', '2025-03-31', '2025-03-05'),
('MTJ1', '2025-04-30', '2025-04-04'),
('MTJ1', '2025-05-31', '2025-05-05'),
('MTJ1', '2025-06-30', '2025-06-04'),
('COI2', '2024-08-31', '2024-07-23'),
('COI2', '2024-09-30', '2024-08-22'),
('COI2', '2024-10-31', '2024-09-22'),
('COI2', '2024-11-30', '2024-10-22'),
('COI2', '2024-12-31', '2024-11-22'),
('COI2', '2025-01-31', '2024-12-23'),
('COI2', '2025-02-28', '2025-01-20'),
('COI2', '2025-03-31', '2025-02-20'),
('COI2', '2025-04-30', '2025-03-22'),
('COI2', '2025-05-31', '2025-04-22'),
('COI2', '2025-06-30', '2025-05-22')
    ) AS t(SiteName, PeriodEnd, PeriodStart)
)
SELECT
    s.SiteName,
    p.ProductKey,
    p.DressMass,
    CONVERT(date, hp.TapTime) AS TapDate,
    pm.PeriodEnd,
    pm.PeriodStart,
    hp.CastQty
FROM PowerBI.HeatProducts hp
LEFT JOIN PowerBI.Products p ON hp.skProductId = p.skProductId
LEFT JOIN PowerBI.Site s ON hp.SkSiteId = s.skSiteId
LEFT JOIN PeriodMap pm
    ON s.SiteName = pm.SiteName
    AND CONVERT(date, hp.TapTime) BETWEEN pm.PeriodStart AND pm.PeriodEnd

ORDER BY hp.TapTime
"""
df_test = pd.read_sql(query_epicor, engine_epicor)

print(df_test)
# ...existing code...

      SiteName    ProductKey  DressMass     TapDate PeriodEnd PeriodStart  \
0         WOD1         AE316       37.4        None      None        None   
1         COI2      BK57548A     4249.0        None      None        None   
2         WUN1         B300C       30.2        None      None        None   
3         COI2      BK54799A     1426.0        None      None        None   
4         WOD1         AE316       37.4        None      None        None   
...        ...           ...        ...         ...       ...         ...   
35884     WOD1       T710LGA       58.2  2025-07-01      None        None   
35885     WOD1      T710SASA       64.8  2025-07-01      None        None   
35886     WOD1       T710WC2        6.4  2025-07-01      None        None   
35887     WOD1     T710WC2XH       15.0  2025-07-01      None        None   
35888     WUN1  BK-CST04047C       28.0  2025-10-17      None        None   

       CastQty  
0         14.0  
1          1.0  
2          3.0  
3      

In [123]:
df_epicor = pd.read_sql(query_epicor, engine_epicor)
print(df_epicor.head(10))

  SiteName    ProductKey    DressMass TapDate PeriodEnd PeriodStart  CastQty
0     MTJ1  1664-301-01D   822.362296    None      None        None      1.0
1     COI2      BK62260A  3324.000000    None      None        None      1.0
2     WUN1         B300C    30.200000    None      None        None      3.0
3     COI2      BK62260A  3324.000000    None      None        None      2.0
4     MTJ1  1664-301-01D   822.362296    None      None        None      1.0
5     COI2      BK62260A  3324.000000    None      None        None      2.0
6     WUN1       D01777C    19.880000    None      None        None     69.0
7     COI2  1980-106-01B  5162.000000    None      None        None      1.0
8     MTJ1  1664-301-01D   822.362296    None      None        None      2.0
9     COI2       BK58662  1556.727740    None      None        None      4.0


GitHub Copilot
Here’s how you can approach this in Python and SQL, step by step:

1. Connect to the new server
2. Prepare your product list and date ranges
3. Build the SQL query
4. Fetch and append the data

In [124]:
df_epicor = pd.read_sql(query_epicor, engine_epicor)
df_epicor.rename(columns={'PeriodEnd': 'DateValue'}, inplace=True)
df_epicor.rename(columns={'CastQty': 'WIPQty'}, inplace=True)
df_epicor.rename(columns={'SiteName': 'WarehouseCode'}, inplace=True)
# ...existing code...

# Define the columns you want to keep
relevant_columns = [
    'WarehouseCode', 'SiteName', 'ProductKey', 'DateValue', 'TapDate',
    'PeriodEnd', 'PeriodStart', 'WIPQty', 'DressMass', 'WIPMass', 'CastQty', 'TradingName'
]

# Select only those columns if they exist in each DataFrame
df = df[[col for col in relevant_columns if col in df.columns]]
df_epicor = df_epicor[[col for col in relevant_columns if col in df_epicor.columns]]

# Concatenate
df_final = pd.concat([df, df_epicor], ignore_index=True)

# ...existing code...

In [125]:
# ...existing code...

# Define the trading name mapping
trading_name_map = {
    '12A': 'Xuzhou', 'XUZ1': 'Xuzhou',
    'H53': 'Mt Joli', 'MTJ1': 'Mt Joli',
    'M61': 'Merlimau', 'MER1': 'Merlimau',
    'COI2': 'Coimbatore', 'I92': 'Coimbatore',
    '235': 'Wodonga', 'WOD1': 'Wodonga'
}

# Ensure TradingName is added to both dataframes
if 'WarehouseCode' in df.columns:
    df['TradingName'] = df['WarehouseCode'].map(trading_name_map)
else:
    df['TradingName'] = None

if 'WarehouseCode' in df_epicor.columns:
    df_epicor['TradingName'] = df_epicor['WarehouseCode'].map(trading_name_map)
else:
    df_epicor['TradingName'] = None

# Concatenate and reorder columns if needed
df_final = pd.concat([df, df_epicor], ignore_index=True)

# Optional: move TradingName to a specific position (e.g., after WarehouseCode)
cols = list(df_final.columns)
if 'TradingName' in cols and 'WarehouseCode' in cols:
    cols.insert(cols.index('WarehouseCode') + 1, cols.pop(cols.index('TradingName')))
    df_final = df_final[cols]

# Export to CSV
df_final.to_csv('final_wip_data_export.csv', index=False)

# ...existing code...

In [103]:
print(df.head(10))
print(df_epicor.head(10))
print(df_final.head(10))

  WarehouseCode    ProductKey   DateValue  WIPQty  DressMass    WIPMass  \
0           H53  1733-204-08F  2023-03-31    29.0    2992.80   86.79120   
1           H53  1733-204-08F  2023-03-31     2.0    2992.80    5.98560   
2           H53  1733-204-08F  2023-03-31     1.0    2992.80    2.99280   
3           H53  1733-204-08F  2023-03-31    28.0    2992.80   83.79840   
4           H53  1733-204-04F  2023-03-31    44.0    2293.36  100.90784   
5           H53   1711-05-01D  2023-03-31    16.0     133.81    2.14096   
6           H53   1711-05-01D  2023-03-31     1.0     133.81    0.13381   
7           H53   1711-05-01D  2023-03-31    34.0     133.81    4.54954   
8           H53   1711-05-01D  2023-03-31     1.0     133.81    0.13381   
9           H53  1733-204-04F  2023-03-31    13.0    2293.36   29.81368   

  TradingName  
0     Mt Joli  
1     Mt Joli  
2     Mt Joli  
3     Mt Joli  
4     Mt Joli  
5     Mt Joli  
6     Mt Joli  
7     Mt Joli  
8     Mt Joli  
9     Mt Joli 